In [1]:
import os
from dotenv import load_dotenv
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import GutenbergLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.evaluation.qa import QAEvalChain

In [2]:
load_dotenv()

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
azure_deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")

# Vérification que la clé API LangSmith est définie
if not os.getenv("LANGCHAIN_API_KEY"):
    raise ValueError("La clé API LangSmith n'est pas définie")

langchain_project = os.getenv("LANGCHAIN_PROJECT")

In [3]:
embedding_model = AzureOpenAIEmbeddings(
    openai_api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_api_endpoint,
    deployment="text-embedding-3-large",
    openai_api_version="2023-05-15",
    chunk_size=500
)

llm = AzureChatOpenAI(
    api_key=azure_openai_api_key,
    api_version="2023-05-15",
    azure_endpoint=azure_openai_api_endpoint,
    deployment_name=azure_deployment_name,
    temperature=0
)

/tmp/ipykernel_1519/3092749887.py:1: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embedding_model = AzureOpenAIEmbeddings(
/tmp/ipykernel_1519/3092749887.py:9: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


In [4]:
from langchain.vectorstores import FAISS

vectorstore_path = "book_index"  # On reprend le vectorstore sauvegardé dans le premier notebook

if os.path.exists(vectorstore_path):
    vectorstore = FAISS.load_local(
        vectorstore_path,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  
    )
    print(f"Vectorstore chargé depuis {vectorstore_path}.")
else:
    print(f"Le vectorstore n'a pas été trouvé à l'emplacement {vectorstore_path}.")

Vectorstore chargé depuis book_index.


In [5]:
retriever = vectorstore.as_retriever()

In [6]:
PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Vous êtes un assistant littéraire expert.

    Utilisez les informations suivantes pour répondre à la question de manière précise et concise.

    Informations :
    {context}

    Question :
    {question}

    Réponse :
    """
)

In [7]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT}
)

In [12]:
def evaluate_with_langsmith(qa_chain, examples):
    eval_chain = QAEvalChain.from_llm(llm)
    predictions = []

    for example in examples:
        answer = qa_chain.run(example["question"])
        predictions.append({"answer": answer})

    graded_outputs = eval_chain.evaluate(
        examples=examples,
        predictions=predictions,
        question_key="question",
        answer_key="answer",
        prediction_key="answer"
    )
    return graded_outputs, predictions

In [20]:
examples = [
    {
        "question": "Quels sont les thèmes principaux du livre 'Les Misérables' ?",
        "answer": "Les thèmes principaux incluent la justice sociale, la rédemption, l'amour, et la pauvreté."
    },
    {
        "question": "Qui sont les personnages principaux dans le résumé du livre 'Le Roi des Étudiants' ?",
        "answer": "Les personnages principaux sont Paul Champfort et ses camarades de lycée"
    },
    {
        "question": "Quels sont les thèmes principaux du livre 'Les fleurs du mal' ?",
        "answer": "Les thèmes principaux sont la mélancolie, la beauté, la société, et la lutte morale."
    },
    {
        "question": "Résume moi 'Fairy Tales from the Arabian Nights' ?",
        "answer": "C'est une collection de contes fantastiques inspirés du folklore moyen-oriental dont Les histoires explorent des thèmes de l'amour, de la magie et de l'aventure"
    },
    {
        "question": "De quoi parle 'The Japanese twins'?",
        "answer" : "C'est un roman pour enfants.L'histoire suit Taro et Take, des jumeaux de cinq ans vivant dans une maison japonaise pittoresque avec leurs parents, leur grand-mère et leur petit frère. Le livre explore leur vie quotidienne, leurs expériences culturelles et les aventures innocentes de l'enfance."
    }
]

In [18]:
# Exécuter l'évaluation
graded_outputs, predictions = evaluate_with_langsmith(qa_chain, examples)

/home/yaskck/.pyenv/versions/3.10.15/envs/langchain_env/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:173: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


In [21]:
# Résultats
for idx, result in enumerate(graded_outputs):
    print("=" * 50)
    print(f"Result dict: {result}")
    print(f"Question {idx + 1}: {examples[idx]['question']}")
    print(f"Réponse générée: {predictions[idx]['answer']}")
    print(f"Réponse attendue: {examples[idx]['answer']}")
    print(f"Évaluation: {result['results']}")
    print("=" * 50)

Result dict: {'results': 'CORRECT'}
Question 1: Quels sont les thèmes principaux du livre 'Les Misérables' ?
Réponse générée: Les thèmes principaux de "Les Misérables" sont la justice, la moralité, la rédemption, et la lutte entre le bien et le mal. Le roman explore également les problèmes sociaux, notamment la pauvreté et la marginalisation, à travers les vies de ses personnages, en particulier Jean Valjean.
Réponse attendue: Les thèmes principaux incluent la justice sociale, la rédemption, l'amour, et la pauvreté.
Évaluation: CORRECT
Result dict: {'results': 'INCORRECT'}
Question 2: Qui sont les personnages principaux dans le résumé du livre 'Le Roi des Étudiants' ?
Réponse générée: Les personnages principaux dans le résumé du livre "Le Roi des Étudiants" sont Paul Champfort et un groupe d'étudiants en médecine à Québec.
Réponse attendue: Les personnages principaux sont Paul Champfort et ses camarades de lycée
Évaluation: INCORRECT
Result dict: {'results': 'CORRECT'}
Question 3: Quel

## Pour voir les métriques : 

##### Grâce à la clé API langsmith et grâce au nom du projet défini dans le .env, on va pouvoir se connecter sur la plateforme "smith.langchain.com" et voir les runs et les métriques

##### Voir dossier "langSmith_screens qui contient quelques captures des métriques

##### Un csv avec l'ensemble des métriques peut également être exporté depuis le site pour des analyses plus poussées

In [22]:
import pandas as pd

langsmith_dataset = pd.read_csv("langSmith_screens/langsmith_export.csv")

langsmith_dataset.head()

,input_input_list,input_query,output_outputs,output_result
0,"[{""query"": ""Quels sont les thèmes principaux d...",NaN,"[{""results"": ""CORRECT""}, {""results"": ""INCORREC...",NaN
1,NaN,De quoi parle 'The Japanese twins'?,NaN,"""The Japanese Twins"" de Lucy Fitch Perkins est..."
2,NaN,Résume moi 'Fairy Tales from the Arabian Night...,NaN,"""Fairy Tales from the Arabian Nights"" par E. D..."
3,NaN,Quels sont les thèmes principaux du livre 'Les...,NaN,"""Les Fleurs du mal"" de Charles Baudelaire expl..."
4,NaN,Qui sont les personnages principaux dans le ré...,NaN,Les personnages principaux dans le résumé du l...


In [23]:
langsmith_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   input_input_list  2 non-null      object
 1   input_query       11 non-null     object
 2   output_outputs    2 non-null      object
 3   output_result     11 non-null     object
dtypes: object(4)
memory usage: 544.0+ bytes


##### Et ainsi de suite en fonction de ce que l'on souhaite afficher, nettoyer, analyser, etc.